# IBM DATASCIENCE CAPSTONE PROJECT 

### BATTLE OF THE NEIGHBORHOODS


Before we get the data and start exploring it, let's download all the dependencies that we will need.

#### Import necessary Libraries

In [1]:
#Import libraries
import pandas as pd #pandas
import matplotlib.pyplot as plt #matplotlib
import numpy as np #numpy
import seaborn as sns #seaborn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
%matplotlib inline
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

### 1. Download and Explore Dataset

First we retrieve the data from wikipedia and then perfrom Webscraping using pandas and beautiful soup

In [2]:
#send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell


In [5]:

# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [6]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [7]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [8]:
# print the number of rows
toronto_df_grouped.shape

(103, 3)

#### Loading of Geo spatial data

In [9]:
#load geospatial data
csv_path = 'https://cocl.us/Geospatial_data'
df = pd.read_csv(csv_path)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#check the number of rows 
df.shape

(103, 3)

In [11]:
#rename postal code header
df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
toronto_df_grouped['Neighborhood'] = toronto_df_grouped['Neighborhood'].str.replace(' /', ',')
toronto_df_grouped['PostalCode'] = toronto_df_grouped['PostalCode'].str.replace('\n', '')
toronto_df_grouped['Borough'] = toronto_df_grouped['Borough'].str.replace('\n', '')
toronto_df_grouped.head() 

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
#we can check the equality condition of dataframes before merging
toronto_df_grouped.iloc[0,0] == df.iloc[0,0]

True

In [14]:
#Now we can merge both dataframes
toronto_df_new2 = pd.merge(left=df, right=toronto_df_grouped, left_on='PostalCode', right_on='PostalCode')
toronto_df_new2.head() 

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [15]:
toronto_df_new = toronto_df_grouped.merge(df, on="PostalCode", how='left')
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
# merge two table on the column "PostalCode"
toronto_df_new2 = pd.merge(left=df, right=toronto_df_grouped, left_on='PostalCode', right_on='PostalCode')


result = pd.concat([toronto_df_grouped, df], axis = 1, sort=False)
result

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


#### Here we import Folium 

In [18]:
import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [19]:
print('The dataframe has {} boroughs and {} Neighborhood.'.format(
        len(result['Borough'].unique()),
        result.shape[0]    )
)

The dataframe has 10 boroughs and 103 Neighborhood.


In [21]:
# to get longitude and latitude for Toronto
address = 'Scarborough, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough Toronto are 43.773077, -79.257774.


In [52]:
# create map of Scarborough Toronto using latitude and longitude values
map_scarb = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format('Neighborhood','Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarb)  
    
map_scarb

### Using Foursquare API to retrieve venues

In [24]:
CLIENT_ID = 'HTZWOICSLARWS54KJTVCSFOYM20WM5P5CGUESFUL0WWYCSGE' # your Foursquare ID
CLIENT_SECRET = 'NY5NHZAFTGIJEWVXO3VPNL5UKFKINB24DJS4QOQE5NSGQIBQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HTZWOICSLARWS54KJTVCSFOYM20WM5P5CGUESFUL0WWYCSGE
CLIENT_SECRET:NY5NHZAFTGIJEWVXO3VPNL5UKFKINB24DJS4QOQE5NSGQIBQ


Here we retrieve the data scraped and merged

In [25]:
scr_data = result[result['Borough'] == 'Scarborough'].reset_index(drop=True)
scr_data.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


#### Testing Foursqaure API connection

Here Foursqaure is used to retrive the location data on Malvern Rouge Scarborough

In [27]:
address = 'Scarborough, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [28]:
scr_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [29]:
mav_latitude = scr_data.loc[0, 'Latitude'] # neighborhood latitude value
mav_longitude = scr_data.loc[0, 'Longitude'] # neighborhood longitude value

mav_name = scr_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(mav_name, 
                                                               mav_latitude, 
                                                               mav_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [30]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    mav_latitude, 
    mav_longitude, 
    radius, 
    LIMIT)

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eff41d6da803b65d25aa9bb'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### Using Foursquare API to pull the list of top 100 venues within 500 meters radius

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Converting venue list to dataframe

In [36]:
scr_venues = getNearbyVenues(names=scr_data['Neighborhood'],
                                   latitudes=scr_data['Latitude'],
                                   longitudes=scr_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [37]:
print(scr_venues.shape)
scr_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [38]:
scr_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [39]:
print('There are {} uniques categories.'.format(len(scr_venues['Venue Category'].unique())))

There are 56 uniques categories.


##### using pandas and one hot coding to form new dataframe

In [45]:
# one hot encoding
scr_onehot = pd.get_dummies(scr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scr_onehot['Neighborhood'] = scr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scr_onehot.columns[-1]] + list(scr_onehot.columns[:-1])
scr_onehot = scr_onehot[fixed_columns]

scr_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,...,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [46]:
scr_onehot.shape

(94, 57)

In [47]:
scr_grouped = scr_onehot.groupby('Neighborhood').mean().reset_index()
scr_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,...,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.200000,0.000000,0.00,0.0000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.00,0.0000,0.000000,0.000000
2,Cedarbrae,0.000000,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.0000,...,0.083333,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.083333,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.333333,0.000000,0.00,0.0000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.0000,...,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.10,0.0000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.125000,0.000000,0.0000,...,0.125000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000


Calculating the mean frequency

In [48]:
num_top_venues = 5

for hood in scr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scr_grouped[scr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2                     Lounge   0.2
3             Breakfast Spot   0.2
4               Skating Rink   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.11
1       Thai Restaurant  0.11
2                Bakery  0.11
3                  Bank  0.11
4                Lounge  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                 venue  freq
0          Pizza Place  0.17
1             Pharmacy  0.08
2  Rental Car Location  0.08
3          Gas Station  0.08
4  Fried Chicken Joint  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1       

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

New dataframe

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scr_grouped['Neighborhood']

for ind in np.arange(scr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,College Stadium
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Rental Car Location
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
